In [30]:
import pandas as pd

In [31]:
campione = pd.read_excel("./campione_400.xlsx")
print("Numero Coppie nel Campione:", len(campione))
print("Numero Coppie 'Matching' nel Campione:", len(campione[campione['Label'] == 1]))
print("Numero Coppie 'Non Matching' nel Campione:", len(campione[campione['Label'] == 0]))

Numero Coppie nel Campione: 400
Numero Coppie 'Matching' nel Campione: 173
Numero Coppie 'Non Matching' nel Campione: 227


# Valutazione del Blocking basato su Regole estratte da ChatGPT con Few-Shot Learning
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Blocco
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Blocco
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Blocco
- True Positive -> Coppie 'Matching' che finiscono nello stesso Blocco

In [25]:
schema_mediato = pd.read_csv("./../datasets/MediatedSchemaSemicolon.csv", sep=';')
schema_mediato = schema_mediato.sample(n=81706, random_state=24)

In [33]:
dictionary = {}
with open("./../2 - Blocking/blockingRulesGPT/clustering_full_24.txt") as file:
    for line in file:
        chiave, valore = line.split(':')
        chiave = int(chiave)
        valore = int(valore[1:-1])
        dictionary[chiave] = valore

dictionary_ordinato = dict(sorted(dictionary.items(), key=lambda x: x[0]))

In [34]:
block = list(dictionary_ordinato.values())
schema_mediato['block'] = block

In [35]:
tn, fp, fn, tp = list(), list(), list(), list()

for i in range(0, len(campione)):
    row = campione.iloc[i]
    first, second, true_label = row['Riga_SX'], row['Riga_DX'], row['Label']

    first_block = schema_mediato.loc[first]['block']
    second_block = schema_mediato.loc[second]['block']

    computed_label = None
    if first_block == second_block:
        computed_label = 1
    else:
        computed_label = 0
    
    if true_label == 0 and computed_label == 0:
        tn.append((first, second))
    if true_label == 0 and computed_label == 1:
        fp.append((first, second))
    if true_label == 1 and computed_label == 0:
        fn.append((first, second))
    if true_label == 1 and computed_label == 1:
        tp.append((first, second))

In [36]:
recall = len(tp) / (len(tp) + len(fn))
precision = len(tp) / (len(tp) + len(fp))
f_measure = (2 * precision * recall) / (precision + recall)

print("Recall:", recall)
print("Precision:", precision)
print("F-Measure:", f_measure)

Recall: 0.6820809248554913
Precision: 0.8805970149253731
F-Measure: 0.7687296416938111


# Valutazione del Pairwise Matching
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Clusterino
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Clusterino
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Clusterino
- True Positive -> Coppie 'Matching' che finiscono nello stesso Clusterino

In [ ]:
## DA ULTIMARE!